In [2]:

from PIL import Image
from tqdm import tqdm 
import os 
import numpy as np 


In [ ]:

parent_folder = "/media/darian/Storage_0/Ocular_datasets/ocular_datasets/SBVPI/3000x1700"

height, width = 400, 300
destination_folder = f"SBVPI_{width}x{height}/images/"
os.makedirs(destination_folder, exist_ok=True)

original_segments = ["sclera", "iris", "pupil", "canthus", "vessels", "eyelashes", "periocular"]

subfolders = os.listdir(os.path.join(parent_folder))
for sub in tqdm(subfolders):    
    for img_name in os.listdir(os.path.join(parent_folder, sub)):
        #print(img_name.split("_")[-1])
        if img_name.split("_")[-1].split(".")[0] in original_segments:
            continue 
        
        img_path = os.path.join(parent_folder, sub, img_name)
        img = Image.open(img_path)
        # if np.asarray(img).shape[2] == 4: 
        #     print("wow")
        #     print(img_path)
        # continue 
        img = img.resize((height, width))
        target = os.path.join(destination_folder, f"{sub}_{img_name}").replace(".jpg", ".png")

        img.save(target,  quality=100, subsampling=0)
        # break

 95%|█████████▍| 52/55 [01:04<00:04,  1.47s/it]

wow
/media/darian/Storage_0/Ocular_datasets/ocular_datasets/SBVPI/3000x1700/34/34L_r_3.png


 98%|█████████▊| 54/55 [01:06<00:01,  1.42s/it]

wow
/media/darian/Storage_0/Ocular_datasets/ocular_datasets/SBVPI/3000x1700/35/35R_u_4.png


100%|██████████| 55/55 [01:07<00:00,  1.24s/it]


In [20]:
desired_segments = ["sclera"]

target_folder = f"SBVPI_{width}x{height}"
target_label_folder = f"{target_folder}/labels/"
os.makedirs(target_label_folder, exist_ok=True)

mask_files_folder = f"{parent_folder}"
# mask_files = []
mask_file_dict = {}
for sub in os.listdir(mask_files_folder):
    for mask_file in os.listdir(os.path.join(mask_files_folder, sub)):
        if mask_file.split("_")[-1].split(".")[0] not in desired_segments:
            continue 
        
        img_id = sub + "/" + "_".join(mask_file.split(".png")[0].split("_")[:-1])
        img_mask_name = mask_file.split("/")[-1].split(".png")[0].split("_")[-1]
        
        if img_id in mask_file_dict:
            mask_file_dict[img_id].append(img_mask_name)

        else: 
            mask_file_dict[img_id] = [img_mask_name]
            
        # img_id = img_path.split("/")[-1].split(".jpg")[0]#.split("_")[:-1]
        # mask_file_dict[img_id] = []

print(mask_file_dict)


how_many_missing = 0 

for x, img_path in tqdm(enumerate(mask_file_dict.keys())):
    
    img = None 
    num_segments = 0
    for seg_id, seg in enumerate(desired_segments):
        num_segments += 1

        mask_name = mask_files_folder + "/"+  img_path + "_" +  seg + ".png"

        tmp_img = Image.open(mask_name)
        tmp_img = tmp_img.convert("L")
        tmp_img = np.array(tmp_img)
        tmp_img = (tmp_img / 255) * (seg_id+1)
    
        tmp_img = Image.fromarray(tmp_img)
        tmp_img = tmp_img.resize((height, width), resample= Image.NEAREST)
        tmp_img = np.array(tmp_img)
        
        try:
            img += tmp_img
            img = np.clip(img, 0, (seg_id+1))
        except:
            img = tmp_img 

    #print(np.unique(img))
    img_new = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8) 
    img_new[img == 1] = [255, 0, 0] # red
    img_new[img == 2] = [0, 255, 0]
    img_new[img == 3] = [0, 0, 255]
    
    #print(np.unique(img_new))
    img_new = Image.fromarray(img_new)
    
    tar_path = os.path.join(target_label_folder, f"{img_path.replace('/', '_')}.png")
    img_new.save(tar_path,  quality=100, subsampling=0) # TODO care for quality! 
    # break
 
print(how_many_missing)

{'44/44L_r_1': ['sclera'], '44/44R_l_1': ['sclera'], '44/44R_l_3': ['sclera'], '44/44L_l_1': ['sclera'], '44/44L_s_3': ['sclera'], '44/44R_r_3': ['sclera'], '44/44L_u_5': ['sclera'], '44/44L_r_2': ['sclera'], '44/44L_u_1': ['sclera'], '44/44L_s_4': ['sclera'], '44/44L_u_3': ['sclera'], '44/44R_r_4': ['sclera'], '44/44R_r_1': ['sclera'], '44/44R_s_3': ['sclera'], '44/44L_u_2': ['sclera'], '44/44R_l_2': ['sclera'], '44/44L_r_4': ['sclera'], '44/44R_s_1': ['sclera'], '44/44L_u_4': ['sclera'], '44/44R_u_1': ['sclera'], '44/44L_l_2': ['sclera'], '44/44R_s_4': ['sclera'], '44/44L_r_3': ['sclera'], '44/44R_u_3': ['sclera'], '44/44L_s_1': ['sclera'], '44/44L_l_4': ['sclera'], '44/44L_s_2': ['sclera'], '44/44R_u_2': ['sclera'], '44/44L_l_3': ['sclera'], '44/44R_r_2': ['sclera'], '44/44R_u_4': ['sclera'], '44/44R_s_2': ['sclera'], '44/44R_l_4': ['sclera'], '2/2R_u_1': ['sclera'], '2/2L_s_1': ['sclera'], '2/2R_l_2': ['sclera'], '2/2R_r_1': ['sclera'], '2/2R_r_2': ['sclera'], '2/2L_l_1': ['sclera'

1840it [01:22, 22.38it/s]

0


In [24]:
# Remove samples that do not have a label
sub = "images"
label_sub = "labels"

dst_folder = "SBVPI_300x400"
filenames =  os.listdir(os.path.join(dst_folder, sub))
label_filenames = os.listdir(os.path.join(dst_folder, label_sub))

for file in (filenames):
    if file not in label_filenames:
        print(file)
        os.remove(os.path.join(dst_folder, sub, file))



2_2R_l_4.png
2_2R_u_3.png
2_2R_r_3.png
2_2R_u_4.png
2_2R_l_3.png
2_2R_s_3.png
2_2L_s_3.png
2_2L_r_3.png
2_2L_u_3.png
2_2R_s_4.png
2_2L_u_4.png
2_2L_s_4.png
2_2L_r_4.png
2_2L_l_4.png
2_2R_r_4.png
2_2L_l_3.png
